In [62]:
import os
import numpy as np
import torch
import torch
from torchvision import transforms,models,datasets
import torch.nn.functional as F
from rich import print
from torch import nn, optim
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader, RandomSampler, random_split, Dataset
from tqdm import tqdm
from torchvision import transforms as T

from PIL import Image
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pydicom


In [63]:
def get_resnet34():

    # Load the pre-trained ResNet-18 model
    resnet34 = models.resnet34(pretrained=True)
    new_conv1 = nn.Conv2d(
    in_channels=1,
    out_channels=resnet34.conv1.out_channels,
    kernel_size=resnet34.conv1.kernel_size,
    stride=resnet34.conv1.stride,
    padding=resnet34.conv1.padding,
    bias=resnet34.conv1.bias is not None
)

    # Copy the weights from the original conv1 layer, averaging across the input channels
    new_conv1.weight.data = resnet34.conv1.weight.data.mean(dim=1, keepdim=True)

    # Replace the original conv1 layer with the new one
    resnet34.conv1 = new_conv1


    num_ftrs = resnet34.fc.in_features  # Get the number of input features to the original fc layer
    resnet34.fc = nn.Sequential(nn.Flatten(),
                nn.Linear(512, 128),
                nn.ReLU(),
                nn.Dropout(0.2),
                nn.Linear(128, 1),
                nn.Sigmoid())
    return resnet34




In [64]:
# sample_img_path = "/kaggle/input/segmentation-new/Segmentation_MRCNN/Pancreas_slice/pancreas_001.dcm"
# sample_img = pydicom.dcmread(sample_img_path).pixel_array

In [65]:
# sample_img = torch.tensor(sample_img)
# sample_img = sample_img.unsqueeze(0).unsqueeze(0)
# sample_img.size()

In [66]:
# resnet34 = get_resnet34()
# features = resnet34(sample_img.float())


In [67]:
# features.shape

In [68]:
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        
        def conv_block(in_channels, out_channels):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
            )

        def up_conv_block(in_channels, out_channels):
            return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        
        # Encoder
        self.encoder1 = conv_block(in_channels, 32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.encoder2 = conv_block(32, 64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.encoder3 = conv_block(64, 128)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.encoder4 = conv_block(128, 256)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.bottleneck = conv_block(256, 512)
        
        # Decoder
        self.upconv4 = up_conv_block(512, 256)
        self.decoder4 = conv_block(512, 256)
        
        self.upconv3 = up_conv_block(256, 128)
        self.decoder3 = conv_block(256, 128)
        
        self.upconv2 = up_conv_block(128, 64)
        self.decoder2 = conv_block(128, 64)
        
        self.upconv1 = up_conv_block(64, 32)
        self.decoder1 = conv_block(64, 32)
        
        self.output_conv = nn.Conv2d(32, out_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

        
    def forward(self, x):
        # Encoder
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))
        
        # Bottleneck
        bottleneck = self.bottleneck(self.pool4(enc4))
        
        # Decoder
        dec4 = self.upconv4(bottleneck)
        dec4 = self.decoder4(torch.cat((dec4, enc4), dim=1))
        
        dec3 = self.upconv3(dec4)
        dec3 = self.decoder3(torch.cat((dec3, enc3), dim=1))
        
        dec2 = self.upconv2(dec3)
        dec2 = self.decoder2(torch.cat((dec2, enc2), dim=1))
        
        dec1 = self.upconv1(dec2)
        dec1 = self.decoder1(torch.cat((dec1, enc1), dim=1))
        
        first = self.output_conv(dec1)
        output = self.sigmoid(first)
        return output


In [75]:
# DataLoader Code for U-Net-32
class SegmentationDataset(Dataset):
    def __init__(self , images , masks):
        self.imgs = images
        self.masks = masks
        
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        image = pydicom.dcmread("/kaggle/input/segmentation-new/Segmentation_MRCNN/Pancreas_slice/" + self.imgs[idx]).pixel_array
        mask = pydicom.dcmread("/kaggle/input/segmentation-new/Segmentation_MRCNN/Mask/" + self.masks[idx]).pixel_array
        
        mask[mask>1] = 1
        
        image_array_float32 = image.astype(np.float32)

        # Normalize the image array to the range [0, 1]
        image_array_normalized = (image_array_float32 - np.min(image_array_float32)) / (np.max(image_array_float32) - np.min(image_array_float32))


        mask_array_float32 = mask.astype(np.float32)
        
        return T.ToTensor()(image_array_normalized),T.ToTensor()(mask_array_float32)


In [76]:
images = sorted(os.listdir("/kaggle/input/segmentation-new/Segmentation_MRCNN/Pancreas_slice"))
masks = sorted(os.listdir("/kaggle/input/segmentation-new/Segmentation_MRCNN/Mask"))


In [77]:
dataset = SegmentationDataset(images, masks)


In [78]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

In [79]:
num_gpus = torch.cuda.device_count()


In [80]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = UNet(in_channels=1, out_channels=1).cuda()  # 3 output channels for multi-class segmentation
model = nn.DataParallel(model, device_ids=[i for i in range(num_gpus)])  


In [46]:
def dice_loss(pred_mask,true_mask):
    pred_mask[pred_mask>0.45]=1
    pred_mask[pred_mask<0.45]=0
    intersection = len(torch.nonzero(pred_mask * true_mask))
    sum_regions = len(torch.nonzero(pred_mask)) + len(torch.nonzero(true_mask))
    dice = (2 * intersection) / (sum_regions)
#     print(pred_mask.shape)
#     print(true_mask.shape)
#     print(dice)
    return torch.tensor(1-dice,requires_grad=True)

In [50]:
# Define the loss function and optimizer
criterion = nn.BCELoss().cuda()  # Ignore background in loss calculation
optimizer = optim.Adam(model.parameters(), lr=5e-3)

In [51]:
num_epochs = 25

In [1]:
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}")

    
    for i,(images, masks) in progress_bar:
#         images,masks = images.float(),masks.float()
#         print(masks)
        images, masks = images.cuda(), masks.cuda()

        # Forward pass
        outputs = model(images)
        loss = dice_loss(outputs.squeeze(1), masks.squeeze(1))
        if not loss.requires_grad:
            raise RuntimeError("Loss tensor does not require gradients")

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader)}')

 # Adjust learning rate at specified epochs
#     if (epoch + 1) % 20 == 0:
#         for param_group in optimizer.param_groups:
#             param_group['lr'] *= 0.05

In [2]:
model.eval()
# epoch_loss = 0
with torch.no_grad():
    progress_bar = tqdm(enumerate(val_loader), total=len(val_loader), desc=f"Epoch {1}")
    total_dice = 0
    total_iou = 0
    total_accuracy = 0
    total_precision = 0
    total_recall = 0

    for i,(images, true_mask) in progress_bar:
    #         images,masks = images.float(),masks.float()
    #         print(masks)
        images, true_mask = images.cuda(), true_mask.cuda()

        # Forward pass
        pred_mask = model(images)
    #         print(outputs.shape)

    #     loss = criterion(outputs.squeeze(1), masks.squeeze(1))
        pred_mask = pred_mask.squeeze(1)
        pred_mask[pred_mask>0.45]=1
        pred_mask[pred_mask<0.45]=0
        true_mask = true_mask.squeeze(1)
#         pred_mask2 = pred_mask.cpu().numpy()
#         plt.imshow(pred_mask2[0],cmap="gray")
#         break
        # Calculate Dice coefficient
        intersection = len(torch.nonzero(pred_mask * true_mask))
#         print(intersection)
        sum_regions = len(torch.nonzero(pred_mask)) + len(torch.nonzero(true_mask))
    #             print(union)
        dice = (2 * intersection) / (sum_regions)  # Add epsilon to avoid division by zero
        total_dice += dice

        iou = intersection / (sum_regions-intersection)
        total_iou+=iou


        # Calculate mean accuracy
        length = len(true_mask)*len(true_mask[0])*len(true_mask[0])
        accuracy = torch.sum(pred_mask == true_mask)/length
    #             print(accuracy)
    #             print(len(true_mask[0]))
        total_accuracy += accuracy.item()

# Compute mean values of metrics
num_batches = len(val_loader)*2
mean_dice = total_dice / num_batches
mean_accuracy = total_accuracy / num_batches
mean_iou = total_iou / num_batches

# Print the mean values of the metrics
print(f"Mean Dice Coefficient:{mean_dice:.4f}")
print(f"Mean IoU:{mean_iou:.4f}")
print(f"Mean Pixel Accuracy:{mean_accuracy:.4f}")

In [25]:
torch.save(model.state_dict(), 'model_UNet32_50_weights.pth')


In [81]:
!pip install pretrained-backbones-unet

In [83]:
from backbones_unet.model.unet import Unet
from backbones_unet.utils.dataset import SemanticSegmentationDataset
from backbones_unet.model.losses import DiceLoss
from backbones_unet.utils.trainer import Trainer

In [90]:
model = Unet(
    backbone='resnet50', # backbone network name
    in_channels=1,            # input channels (1 for gray-scale images, 3 for RGB, etc.)
    num_classes=1,            # output channels (number of classes in your dataset)
).cuda()
model = nn.DataParallel(model, device_ids=[i for i in range(num_gpus)])  


In [91]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, 1e-4) 
criterion=DiceLoss().cuda()

In [92]:
epochs=10



In [93]:
# trainer.fit(train_loader, val_loader

In [94]:
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}")

    
    for i,(images, masks) in progress_bar:
#         images,masks = images.float(),masks.float()
#         print(masks)
        images, masks = images.cuda(), masks.cuda()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.squeeze(1), masks.squeeze(1))
        if not loss.requires_grad:
            raise RuntimeError("Loss tensor does not require gradients")

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader)}')

 # Adjust learning rate at specified epochs
#     if (epoch + 1) % 20 == 0:
#         for param_group in optimizer.param_groups:
#             param_group['lr'] *= 0.05

Epoch 1: 100%|██████████| 576/576 [01:52<00:00,  5.13it/s]


Epoch [1/25], Loss: 0.9199051691426171

Epoch 2: 100%|██████████| 576/576 [01:51<00:00,  5.17it/s]


Epoch [2/25], Loss: 0.8761563339374132

Epoch 3: 100%|██████████| 576/576 [01:52<00:00,  5.11it/s]


...

Epoch [23/25], Loss: 0.0475090883879198

Epoch 24: 100%|██████████| 576/576 [01:52<00:00,  5.13it/s]


Epoch [24/25], Loss: 0.04566872720089224

Epoch 25: 100%|██████████| 576/576 [01:52<00:00,  5.11it/s]


Epoch [25/25], Loss: 0.04341397165424294

In [134]:
model.eval()
# epoch_loss = 0
with torch.no_grad():
    progress_bar = tqdm(enumerate(val_loader), total=len(val_loader), desc=f"Epoch {1}")
    total_dice = 0
    total_iou = 0
    total_accuracy = 0
    total_precision = 0
    total_recall = 0

    for i,(images, true_mask) in progress_bar:
    #         images,masks = images.float(),masks.float()
    #         print(masks)
        images, true_mask = images.cuda(), true_mask.cuda()

        # Forward pass
        pred_mask = model(images)
    #         print(outputs.shape)

    #     loss = criterion(outputs.squeeze(1), masks.squeeze(1))
#         pred_mask = pred_mask.squeeze(1)
        pred_mask[pred_mask>0.35]=1
        pred_mask[pred_mask<0.35]=0
        
        pred_mask2 = pred_mask.permute(0,2,3,1).cpu().numpy()
        true_mask2 = true_mask.permute(0,2,3,1).cpu().numpy()
        true_mask = true_mask.squeeze(1)
        pred_mask = pred_mask.squeeze(1)

#         print(pred_mask.shape)
#         print(true_mask.shape)
        
#         plt.imshow(pred_mask2[0],cmap="gray")
#         break
#         # Calculate Dice coefficient
        intersection = len(torch.nonzero(pred_mask * true_mask))
# #         print(intersection)
        sum_regions = len(torch.nonzero(pred_mask)) + len(torch.nonzero(true_mask))
#     #             print(union)
#         dice = (2 * intersection) / (sum_regions)  # Add epsilon to avoid division by zero
        dice = criterion(pred_mask, true_mask)

        total_dice += dice

#         iou = intersection / (sum_regions-intersection)
#         total_iou+=iou

        
        # Calculate mean accuracy
        length = len(true_mask)*len(true_mask[1])*len(true_mask[1])
#         print(length)
#         break
        accuracy = torch.sum(pred_mask == true_mask)/length
#         print(accuracy)
#         print(len(true_mask[0]))
#         break
        total_accuracy += accuracy.item()

# Compute mean values of metrics
num_batches = len(val_loader)*2
mean_dice = total_dice / num_batches
mean_accuracy = total_accuracy / num_batches
# mean_iou = total_iou / num_batches

# Print the mean values of the metrics
print(f"Mean Dice Coefficient:{1-mean_dice:.4f}")
# print(f"Mean IoU:{mean_iou:.4f}")
print(f"Mean Pixel Accuracy:{mean_accuracy:.4f}")

Epoch 1: 100%|██████████| 144/144 [00:17<00:00,  8.34it/s]


Mean Dice Coefficient:0.5197

Mean Pixel Accuracy:0.4949

In [109]:
torch.save(model.state_dict(), 'model_UNet32_25_ResNet50_weights.pth')

